In [ ]:
import os
print(os.getcwd())



/content


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import os

def parse_xml_files(folder_path):
    data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".xml"):
            file_path = os.path.join(folder_path, file_name)
            tree = ET.parse(file_path)
            root = tree.getroot()

            for utterance in root.findall(".//u"):
                speaker = utterance.attrib.get("who", "Unknown")
                text = " ".join([w.text for w in utterance.findall(".//w") if w.text])

                if any(dm in text for dm in ["absolutely", "yeah", "quite", "clearly", "utterly", "proper", "dead", "first", "second", "third", "oh", "indeed", "I mean", "mind you", "erm", "you know", "okay", "anyway", "uh", "mm", "yes", "no", "well", "so", "just", "like", "um", "right", "totally", "maybe", "sort of", "kind of", "perhaps", "I guess", "it seems", "could be", "possibly", "might", "fairly", "almost", "at least", "pretty much", "in a way", "more or less", "to some extent", "roughly", "probably", "relatively", "as far as I can tell", "more likely than not", "sometimes", "definitely", "really", "for sure", "undoubtedly", "certainly", "always", "forever", "completely", "extremely", "without a doubt", "in fact", "of course", "never", "entirely", "surely", "quite literally", "positively", "unequivocally", "to be honest", "by all means", "as a matter of fact"]
):  
                   data.append({"speaker": speaker, "text": text})

    return pd.DataFrame(data)


folder_path = "/"  
dataset = parse_xml_files(folder_path)

dataset.head()

speaker                                               text
0   S0179  could you move slightly ? So that we 're not q...
1   S0058  yeah I was thinking that yeah I 've gone and p...
2   S0179           there 's probably a little more room for
3   S0058                        I 'll just go on that table
4   S0058               oh dear anyway I was gon na tell you

In [ ]:
metadata = pd.read_csv("/content/bnc2014spoken-speakerdata.tsv", delimiter="\t")  # Specify the tab delimiter

metadata = metadata.rename(columns={"S0001": "speaker", "F": "sex"})

metadata = metadata[["speaker", "sex"]]

#  the cleaned metadata
print(metadata.head())

dataset = dataset.merge(metadata[["speaker", "sex"]], on="speaker", how="left")

dataset = dataset[~dataset["sex"].isnull()]


gender_mapping = {"M": 0, "F": 1}
dataset["gender"] = dataset["sex"].map(gender_mapping)


print(dataset.head())


dataset.head()


  speaker sex
0   S0002   F
1   S0003   F
2   S0004   M
3   S0005   F
4   S0006   M
  speaker                                               text sex  gender
0   S0179  could you move slightly ? So that we 're not q...   M     0.0
1   S0058  yeah I was thinking that yeah I 've gone and p...   F     1.0
2   S0179           there 's probably a little more room for   M     0.0
3   S0058                        I 'll just go on that table   F     1.0
4   S0058               oh dear anyway I was gon na tell you   F     1.0


speaker                                               text sex  gender
0   S0179  could you move slightly ? So that we 're not q...   M     0.0
1   S0058  yeah I was thinking that yeah I 've gone and p...   F     1.0
2   S0179           there 's probably a little more room for   M     0.0
3   S0058                        I 'll just go on that table   F     1.0
4   S0058               oh dear anyway I was gon na tell you   F     1.0

In [ ]:
from transformers import BertTokenizer
import torch


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

label_mapping = {'M': 0, 'F': 1}
labels = dataset["sex"].map(label_mapping)

def tokenize_data(texts, labels, max_length=128):
    encodings = tokenizer(
        texts.tolist(),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    )
    encodings["labels"] = torch.tensor(labels.values, dtype=torch.long)
    return encodings

#  Tokenizing the data
texts = dataset["text"]
encodings = tokenize_data(texts, labels)

print("Tokenization complete")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Tokenization complete


In [ ]:
# separate the classes
male_samples = dataset[dataset["gender"] == 0]
female_samples = dataset[dataset["gender"] == 1]

# downsample the majority class (female)
female_downsampled = female_samples.sample(len(male_samples), random_state=42)


balanced_data = pd.concat([male_samples, female_downsampled])

print(balanced_data["gender"].value_counts())


gender
0.0    320133
1.0    320133
Name: count, dtype: int64


In [ ]:
male_samples = balanced_data[balanced_data['gender'] == 0].reset_index(drop=True)
female_samples = balanced_data[balanced_data['gender'] == 1].reset_index(drop=True)

male_samples = male_samples.sample(frac=1, random_state=42).reset_index(drop=True)
female_samples = female_samples.sample(frac=1, random_state=42).reset_index(drop=True)

min_length = min(len(male_samples), len(female_samples))
interleaved_data = pd.concat(
    [male_samples.iloc[:min_length], female_samples.iloc[:min_length]]
).sort_index(kind='merge').reset_index(drop=True)


print(interleaved_data.head(10))

  speaker                                               text sex  gender
0   S0221             what was the first part of the thing ?   M     0.0
1   S0417  I said I 'm not actually I do n't wan na I do ...   F     1.0
2   S0331  I know it 's slightly annoying but it does sto...   M     0.0
3   S0041                                 no bed all weekend   F     1.0
4   S0629  yeah so it 's I I know what that feeling is li...   M     0.0
5   S0432  yeah no one else everyone 's just that --UNCLE...   F     1.0
6   S0635                                               yeah   M     0.0
7   S0653  alright well hang on a minute what were you go...   F     1.0
8   S0355    working on my own I find quite lonely sometimes   M     0.0
9   S0266        did they not suggest a jacket --UNCLEARWORD   F     1.0


In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --no-cache-dir

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import os

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data import Dataset

class DiscourseMarkerDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

def tokenize_data(sentences, labels, tokenizer, max_length=128):
    encodings = tokenizer(sentences, truncation=True, padding=True, max_length=max_length)
    return DiscourseMarkerDataset(encodings, labels)


balanced_data = interleaved_data  
sentences = balanced_data['text'].tolist()
labels = balanced_data['gender'].tolist()


train_size = int(0.8 * len(sentences))
train_sentences, eval_sentences = sentences[:train_size], sentences[train_size:]
train_labels, eval_labels = labels[:train_size], labels[train_size:]


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


train_dataset = tokenize_data(train_sentences, train_labels, tokenizer)
eval_dataset = tokenize_data(eval_sentences, eval_labels, tokenizer)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=32,
    num_train_epochs=3,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(), 
    report_to="none",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


print("Fine-tuning ...")
trainer.train()

print("\nEvaluating model...")

def evaluate_model(dataset, model, tokenizer, device):
    model.eval()
    predictions, true_labels = [], []
    errors = []

    for idx, row in dataset.iterrows():
        text = row['text']
        true_label = row['gender']

        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_label = torch.argmax(logits, dim=1).item()

        predictions.append(predicted_label)
        true_labels.append(true_label)

        if predicted_label != true_label:
            errors.append({
                "text": text,
                "true_label": true_label,
                "predicted_label": predicted_label
            })

    return np.array(predictions), np.array(true_labels), pd.DataFrame(errors)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

eval_data = pd.DataFrame({"text": eval_sentences, "gender": eval_labels})
y_pred, y_true, errors = evaluate_model(eval_data, model, tokenizer, device)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Male", "Female"]))
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

errors.to_csv("bert_fine_tuned_errors_full_dataset.csv", index=False)
print("Misclassified examples saved to 'bert_fine_tuned_errors_full_dataset.csv'")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Fine-tuning on the entire dataset...



Evaluating the fine-tuned model...

Classification Report:
              precision    recall  f1-score   support

        Male       0.63      0.46      0.53     64027
      Female       0.57      0.73      0.64     64027

    accuracy                           0.59    128054
   macro avg       0.60      0.59      0.59    128054
weighted avg       0.60      0.59      0.59    128054


Confusion Matrix:
[[29313 34714]
 [17266 46761]]
Misclassified examples saved to 'bert_fine_tuned_errors_full_dataset.csv'
